In [1]:
#Import Statements
import numpy as np
import mnist_loader

In [2]:
class Network:
    #Construcutor
    def __init__(self,neurons):
        self.layers = len(neurons)
        self.neurons = neurons
        
        self.weights = []
        self.biases = []
        
        #Initialize weights
        for i in range(1,self.layers):
            #layer_weight = None
            rows  = neurons[i]
            cols = neurons[i-1]
            #Creates a numpy array with dimensions rows x cols
            #At the same time, initializes them with random normal distribution b/w 0 and 1
            layer_weight = np.zeros((rows,cols))
            self.weights.append(layer_weight)
                
            layer_bias = np.zeros((rows,1))
            self.biases.append(layer_bias)

In [3]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

In [4]:
def feedforward(network,X):
    
    #Stores the Aj values
    Layer_Activations = []
    
    #Stores the Zj values
    Weighted_Sums = []
    
    for i in range(1,network.layers):
   
        if i==1:
            A = X
        else:
            A  = Layer_Activations[-1]
        
        W = network.weights[i-1]
        B = network.biases[i-1]
        Z = np.dot(W,A) + B
        Weighted_Sums.append(Z)
        A = sigmoid(Z)
        Layer_Activations.append(A)

    return (Weighted_Sums,Layer_Activations)

In [5]:
def test(Output,Target):
    #Output --> (10,Number_of_training_Samples)
    #Target --> (Number_of_traing_Samples)
    
    Prediction = np.argmax(Output,axis=0)
    #Prediction --> (Number_of_training_Samples,)    
    
    accuracy = (sum((Prediction == Target).astype(np.float32))/Target.shape[0])*100.0
    
    return accuracy

In [6]:
def sigmoid_derivative(z):
    #print("sigmoid_derivative =",sigmoid(z)*(1-sigmoid(z)) )
    return sigmoid(z)*(1-sigmoid(z))
    

In [7]:
def cost_gradient(output,label):
    #print("cost_gradient =",output-label)
    return output-label

In [8]:
def backprop(network,delta_output_layer,weighted_sums):
    #Initialize error for all layers
    #delta is the backpropogation error
    
    #Number of output layers --> first layer is an input layer, hence excluded
    n_out = network.layers-1
    
    delta_all_layers = [None]*(n_out)
    
    delta_all_layers[-1] = delta_output_layer
    
    for i in range(n_out-2,-1,-1):
        delta_all_layers[i] = np.dot(network.weights[i+1].transpose(),delta_all_layers[i+1])*sigmoid_derivative(weighted_sums[i])
            
    return delta_all_layers

In [9]:
def update_weights(network,X,layer_activations,delta_all_layers,alpha):
    
    n_weights = network.layers-1
    for i in range(n_weights):
        
        print("shape of delta_all_layers[",i,"]:",delta_all_layers[i].shape)
        print("shape of layer_activations[",i,"].transpose():",delta_all_layers[i].transpose().shape)            
        if i==0:
            layer_input = X
        else:
            layer_input = layer_activations[i-1]
        
        dcdw = np.dot(delta_all_layers[i],layer_input.transpose())
        dcb = np.average(delta_all_layers[i],axis=1).reshape(delta_all_layers[i].shape[0],1)
        print("shape of dcdw:",dcdw.shape)
        print("shape of dcb:",dcb.shape)
        print("shape of network.weights[",i,"]:",network.weights[i].shape)
        
        network.weights[i] -= alpha * dcdw
        network.biases[i] -= alpha * dcb
        
    return

In [10]:
train_data,valid_data,test_data = mnist_loader.load_data_wrapper()
network = Network([784,16,10])
weighted_sums,activations = feedforward(network,train_data[0])
# print(activations[-1])
delta_output_layer = cost_gradient(activations[-1],train_data[1])*sigmoid_derivative(weighted_sums[-1])
delta_all_layers = backprop(network,delta_output_layer,weighted_sums)
# delta_all_layers[::-1]
update_weights(network,train_data[0],activations,delta_all_layers,1)
network.weights[-1]

shape of delta_all_layers[ 0 ]: (16, 50000)
shape of layer_activations[ 0 ].transpose(): (50000, 16)
shape of dcdw: (16, 784)
shape of dcb: (16, 1)
shape of network.weights[ 0 ]: (16, 784)
shape of delta_all_layers[ 1 ]: (10, 50000)
shape of layer_activations[ 1 ].transpose(): (50000, 10)
shape of dcdw: (10, 16)
shape of dcb: (10, 1)
shape of network.weights[ 1 ]: (10, 16)


array([[-2508.5  , -2508.5  , -2508.5  , -2508.5  , -2508.5  , -2508.5  ,
        -2508.5  , -2508.5  , -2508.5  , -2508.5  , -2508.5  , -2508.5  ,
        -2508.5  , -2508.5  , -2508.5  , -2508.5  ],
       [-2415.25 , -2415.25 , -2415.25 , -2415.25 , -2415.25 , -2415.25 ,
        -2415.25 , -2415.25 , -2415.25 , -2415.25 , -2415.25 , -2415.25 ,
        -2415.25 , -2415.25 , -2415.25 , -2415.25 ],
       [-2504.   , -2504.   , -2504.   , -2504.   , -2504.   , -2504.   ,
        -2504.   , -2504.   , -2504.   , -2504.   , -2504.   , -2504.   ,
        -2504.   , -2504.   , -2504.   , -2504.   ],
       [-2487.375, -2487.375, -2487.375, -2487.375, -2487.375, -2487.375,
        -2487.375, -2487.375, -2487.375, -2487.375, -2487.375, -2487.375,
        -2487.375, -2487.375, -2487.375, -2487.375],
       [-2517.625, -2517.625, -2517.625, -2517.625, -2517.625, -2517.625,
        -2517.625, -2517.625, -2517.625, -2517.625, -2517.625, -2517.625,
        -2517.625, -2517.625, -2517.625, -2517.6